## Importing Libraries

In [1]:
from string import Template
import random


import kfp
from kfp import dsl
from kfp.components import func_to_container_op

## Define Kubeflow Pipeline

In [2]:
@func_to_container_op
def show_results(accuracy :float) -> None:
    print(f"Accuracy of neural network: {accuracy}")

@dsl.pipeline(name='demo_pipeline', description='Fits a simple neural network')
def first_pipeline():

    download = kfp.components.load_component_from_file('download_data/download_data.yaml')
    preprocess_data = kfp.components.load_component_from_file('preprocess_data/preprocess_data.yaml')
    train_model = kfp.components.load_component_from_file('train_model/train_model.yaml')

    download_task = download()
    preprocess_data_task = preprocess_data(download_task.output)
    train_model_task = train_model(preprocess_data_task.output)
    
    show_results(train_model_task.outputs['Accuracy'])
    
if __name__ == '__main__':
    file_name = 'pipeline.yaml'
    kfp.compiler.Compiler().compile(first_pipeline, file_name)

## Upload pipeline

In [ ]:
import kfp
# When not specified, host defaults to env var KF_PIPELINES_ENDPOINT.
# This is now equivalent to `client = kfp.Client(host='http://localhost:3000')`
client = kfp.Client(host='<my-host-url>')
print(client.list_experiments())

In [ ]:
import kfp
from kfp.components import create_component_from_func
import kfp.dsl as dsl

def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    return a + b


# create a component using the add function
add_op = create_component_from_func(add)

# compose your pipeline using the dsl.pipeline decorator
@dsl.pipeline(
    name='Addition pipeline',
    description='An example pipeline that performs addition calculations.')
def add_pipeline(
    a='1',
    b='7',
):
    first_add_task = add_op(a, 4)
    second_add_task = add_op(first_add_task.output, b)

# instantiate a client and submit your pipeline with arguments
client = kfp.Client(host='<my-host-url>')
client.create_run_from_pipeline_func(
    add_pipeline, arguments={
        'a': '7',
        'b': '8'
    })

In [ ]:
import kfp
client=kfp.Client()
#filepath = '/home/jovyan/AI-platform/kubeflow/example/' + file_name
filepath = os.getcwd() + file_name
name = 'kubeflow_demo_' + str(random.randint(30000,90000))
print("Uploaded pipeline:", name)
pipeline = client.pipeline_uploads.upload_pipeline(filepath, name=name)